### "XGBoost Ensemble dengan Circular-Entropy Features untuk Eye-Tracking Classification".

In [ ]:
# xgb_rowlevel_push80.py
# Ensembling XGBoost row-level:
# - tetap pakai feature engineering kamu (rolling, circular stats, hist, entropy, dll)
# - bedanya: train banyak base models (variasi scaler / seed / param mix) => soft-voting
# - threshold search dilebarin (0.15..0.85)
# Target: dorong akurasi 80%+

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from xgboost import XGBClassifier

# =========================
# CONFIG
# =========================
CSV_PATH = "truncated_dataset-seqglo.csv"
ROLL_W = 21
USE_SUBSAMPLE = True
SUBSAMPLE_N = 160_000     # sedikit dinaikkan
RANDOM_STATE = 42

# Coba aktifkan GPU kalau ada
TREE_METHOD = "hist"  # ganti ke "gpu_hist" kalau environment mendukung

SCALE_MODES = ["robust", "quantile", "standard"]  # kita ensemble 3 scaler
SEEDS = [42, 7, 1029]                              # 3 seed -> total 3*3 = 9 model
# Dua variasi param ringan (mixA/mixB) untuk diversitas
PARAM_MIXES = [
    dict(max_depth=5, min_child_weight=7, subsample=0.85, colsample_bytree=0.85, gamma=0.25, reg_alpha=0.6, reg_lambda=2.2),
    dict(max_depth=6, min_child_weight=5, subsample=0.80, colsample_bytree=0.80, gamma=0.20, reg_alpha=0.5, reg_lambda=2.6),
]

BASE_KW = dict(
    objective="binary:logistic",
    n_estimators=1200,     # sedikit lebih panjang, lr diturunkan
    learning_rate=0.045,
    tree_method=TREE_METHOD,
    n_jobs=4,
)

# =========================
# Utils
# =========================
def entropy_hist(a, bins_edges):
    a = np.asarray(a)
    a = a[np.isfinite(a)]
    if a.size == 0: return 0.0
    hist, _ = np.histogram(a, bins=bins_edges)
    s = hist.sum()
    if s == 0: return 0.0
    p = hist.astype(float) / s
    p = p[p > 0]
    return float(-(p * np.log(p)).sum())

def hist_feats(a, bins_edges, prefix):
    hist, _ = np.histogram(a, bins=bins_edges)
    total = hist.sum() + 1e-9
    d = {}
    for i, c in enumerate(hist):
        d[f"{prefix}_bin{i}"] = float(c)
        d[f"{prefix}_p{i}"] = float(c) / total
    d[f"{prefix}_sum"] = float(total)
    return d

# =========================
# 1) Load & pre-bins (global)
# =========================
df = pd.read_csv(CSV_PATH).sort_values(["nama", "time"]).reset_index(drop=True)

dx_all = df["gazeX"].diff()
dy_all = df["gazeY"].diff()
mask_new_subject = df["nama"].ne(df["nama"].shift(1))
dx_all = dx_all.mask(mask_new_subject, 0.0)
dy_all = dy_all.mask(mask_new_subject, 0.0)
step_all = np.sqrt(dx_all**2 + dy_all**2).fillna(0.0).values

q = np.quantile(step_all, [0.00, 0.10, 0.25, 0.50, 0.75, 0.90, 0.97, 0.995, 1.00])
step_bins = np.unique(q)
if step_bins.size < 6:
    step_bins = np.linspace(float(step_all.min()), float(step_all.max()+1e-6), 9)

dang_bins = np.linspace(-np.pi, np.pi, 9)
small_thr = np.quantile(step_all, 0.25)
large_thr = np.quantile(step_all, 0.90)

# =========================
# 2) Feature Engineering (sama seperti punyamu)
# =========================
def add_roll_feats(g, w=ROLL_W):
    g = g.copy()

    dx = g["gazeX"].diff(); dy = g["gazeY"].diff()
    dx.iloc[0] = 0.0; dy.iloc[0] = 0.0
    step = np.sqrt(dx**2 + dy**2)

    ang = np.arctan2(dy, dx)
    d_ang = np.diff(ang, prepend=ang.iloc[0])
    d_ang = (d_ang + np.pi) % (2*np.pi) - np.pi

    cos_da = np.cos(d_ang); sin_da = np.sin(d_ang)
    r_mean_cos = pd.Series(cos_da, index=g.index).rolling(w, min_periods=1).mean()
    r_mean_sin = pd.Series(sin_da, index=g.index).rolling(w, min_periods=1).mean()
    r_mrl = np.sqrt(r_mean_cos**2 + r_mean_sin**2)
    r_circ_var = 1.0 - r_mrl

    g["r_std_x"] = g["gazeX"].rolling(w, min_periods=1).std()
    g["r_std_y"] = g["gazeY"].rolling(w, min_periods=1).std()
    g["r_mean_step"] = step.rolling(w, min_periods=1).mean()
    g["r_std_step"]  = step.rolling(w, min_periods=1).std()
    g["r_mean_abs_dang"] = pd.Series(np.abs(d_ang), index=g.index).rolling(w, min_periods=1).mean()
    g["r_std_dang"] = pd.Series(d_ang, index=g.index).rolling(w, min_periods=1).std()
    try:
        g["r_skew_step"] = pd.Series(step, index=g.index).rolling(w, min_periods=1).skew()
        g["r_kurt_step"] = pd.Series(step, index=g.index).rolling(w, min_periods=1).kurt()
    except Exception:
        g["r_skew_step"] = 0.0; g["r_kurt_step"] = 0.0

    g["r_q25_step"] = pd.Series(step, index=g.index).rolling(w, min_periods=1).quantile(0.25)
    g["r_q75_step"] = pd.Series(step, index=g.index).rolling(w, min_periods=1).quantile(0.75)

    disp_x = g["gazeX"] - g["gazeX"].shift(w-1)
    disp_y = g["gazeY"] - g["gazeY"].shift(w-1)
    disp = np.sqrt(disp_x**2 + disp_y**2)
    path_w = pd.Series(step, index=g.index).rolling(w, min_periods=1).sum() + 1e-6
    g["r_straight_ratio"] = (disp / path_w).fillna(0)
    g["bbox_w"] = g["gazeX"].rolling(w, min_periods=1).max() - g["gazeX"].rolling(w, min_periods=1).min()
    g["bbox_h"] = g["gazeY"].rolling(w, min_periods=1).max() - g["gazeY"].rolling(w, min_periods=1).min()

    g["r_entropy_dang"] = pd.Series(d_ang, index=g.index).rolling(w, min_periods=1).apply(
        lambda a: entropy_hist(a, dang_bins), raw=True
    )
    g["r_entropy_step"] = pd.Series(step, index=g.index).rolling(w, min_periods=1).apply(
        lambda a: entropy_hist(a, step_bins), raw=True
    )

    def roll_hist_features(series, bins, pref):
        arr = []
        s = series.values; n = len(s)
        for i in range(n):
            i0 = max(0, i - w + 1)
            win = s[i0:i+1]
            d = hist_feats(win, bins, pref)
            arr.append(d)
        return pd.DataFrame(arr, index=series.index)

    dang_hist_df = roll_hist_features(pd.Series(d_ang, index=g.index), dang_bins, "h_dang")
    step_hist_df = roll_hist_features(pd.Series(step, index=g.index), step_bins, "h_step")
    g = pd.concat([g, dang_hist_df, step_hist_df], axis=1)

    small_mask = (step <= small_thr).astype(float)
    large_mask = (step >= large_thr).astype(float)
    g["r_rate_small"] = pd.Series(small_mask, index=g.index).rolling(w, min_periods=1).mean()
    g["r_rate_large"] = pd.Series(large_mask, index=g.index).rolling(w, min_periods=1).mean()
    g["r_ratio_small_large"] = (g["r_rate_small"] / (g["r_rate_large"] + 1e-6)).replace([np.inf, -np.inf], 0.0)

    g["dx"] = dx; g["dy"] = dy
    g["abs_dx"] = dx.abs(); g["abs_dy"] = dy.abs()
    g["step"] = step
    g["r_mean_cos_dang"] = r_mean_cos
    g["r_mean_sin_dang"] = r_mean_sin
    g["r_mrl_dang"] = r_mrl
    g["r_circvar_dang"] = r_circ_var

    return g

try:
    df = df.groupby("nama", group_keys=False).apply(add_roll_feats, include_groups=False, w=ROLL_W)
except TypeError:
    df = df.groupby("nama", group_keys=False).apply(add_roll_feats, w=ROLL_W)

# bersihin NaN/Inf
for col in df.columns:
    if col in ["nama", "time", "gazeX", "gazeY", "label"]:
        continue
    med = pd.to_numeric(df[col], errors="coerce").replace([np.inf, -np.inf], np.nan).median()
    df[col] = pd.to_numeric(df[col], errors="coerce").replace([np.inf, -np.inf], np.nan).fillna(med)

# =========================
# 3) Features & Label
# =========================
exclude = {"nama", "label"}
feature_cols = [c for c in df.columns if c not in exclude]
X = df[feature_cols].astype(float).copy()
y = (df["label"] == 2).astype(int).values

# =========================
# 4) Split
# =========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

# subsample train imbang
if USE_SUBSAMPLE:
    rng = np.random.RandomState(RANDOM_STATE)
    ix_pos = np.where(y_train == 1)[0]
    ix_neg = np.where(y_train == 0)[0]
    n_each = min(SUBSAMPLE_N // 2, len(ix_pos), len(ix_neg))
    sel = np.concatenate([
        rng.choice(ix_pos, n_each, replace=False),
        rng.choice(ix_neg, n_each, replace=False),
    ])
    X_train_small = X_train.iloc[sel].reset_index(drop=True)
    y_train_small = y_train[sel]
else:
    X_train_small = X_train.reset_index(drop=True)
    y_train_small = y_train

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_small, y_train_small, test_size=0.15, stratify=y_train_small, random_state=RANDOM_STATE
)

# =========================
# 5) Scalers
# =========================
def make_scaler(mode):
    if mode == "standard":
        return StandardScaler()
    elif mode == "robust":
        return RobustScaler()
    elif mode == "quantile":
        return QuantileTransformer(output_distribution="normal", random_state=RANDOM_STATE, subsample=300_000)
    else:
        raise ValueError("unknown scaler")

def apply_scaler(sc, Xdf, fit=False):
    if sc is None: return Xdf.values
    return (sc.fit_transform(Xdf) if fit else sc.transform(Xdf))

# =========================
# 6) Train an ensemble
# =========================
models = []
scalers = []
for mode in SCALE_MODES:
    sc = make_scaler(mode)
    Xtr_s = apply_scaler(sc, X_tr, fit=True)
    Xva_s = apply_scaler(sc, X_val, fit=False)

    for mix in PARAM_MIXES:
        for seed in SEEDS:
            kw = dict(BASE_KW)
            kw.update(mix)
            kw["random_state"] = seed
            clf = XGBClassifier(**kw)
            clf.fit(Xtr_s, y_tr)
            models.append((mode, seed, mix, clf))
    scalers.append((mode, sc))

# =========================
# 7) Validation blend & threshold search (luas)
# =========================
# Kumpulkan proba val dari semua model (soft-vote)
proba_val_all = []
for (mode, seed, mix, clf) in models:
    sc = dict(scalers)[mode]
    Xva_s = apply_scaler(sc, X_val, fit=False)
    proba_val_all.append(clf.predict_proba(Xva_s)[:, 1])

proba_val_blend = np.mean(proba_val_all, axis=0)

ths = np.arange(0.15, 0.851, 0.001)  # jauh lebih luas
best_thr, best_acc, best_auc = 0.5, -1.0, None
for t in ths:
    preds = (proba_val_blend >= t).astype(int)
    acc = accuracy_score(y_val, preds)
    if acc > best_acc:
        best_thr = float(t); best_acc = acc
try:
    best_auc = roc_auc_score(y_val, proba_val_blend)
except Exception:
    best_auc = float("nan")

print(f"[VAL] Ensemble | Acc: {best_acc:.4f} | AUC: {best_auc:.4f} | thr={best_thr:.3f}")

# =========================
# 8) Test with blended models
# =========================
# NOTE: fit scaler on full-train (X_train_small), bukan hanya X_tr
proba_test_all = []
for mode in SCALE_MODES:
    sc = make_scaler(mode)
    sc.fit(X_train_small)  # fit di seluruh train kecil
    Xtest_s = apply_scaler(sc, X_test, fit=False)

    # ambil semua model yang cocok mode-nya
    for (m_mode, seed, mix, clf) in models:
        if m_mode != mode: continue
        # retrain model di full-train kecil dengan scaler yg baru
        kw = dict(BASE_KW); kw.update(mix); kw["random_state"] = seed
        clf_full = XGBClassifier(**kw)
        Xfull_s = apply_scaler(sc, X_train_small, fit=False)
        clf_full.fit(Xfull_s, y_train_small)
        proba_test_all.append(clf_full.predict_proba(Xtest_s)[:, 1])

proba_test_blend = np.mean(proba_test_all, axis=0)
preds_test = (proba_test_blend >= best_thr).astype(int)

acc = accuracy_score(y_test, preds_test)
try:
    auc = roc_auc_score(y_test, proba_test_blend)
except Exception:
    auc = float("nan")
cm = confusion_matrix(y_test, preds_test)
report = classification_report(y_test, preds_test, target_names=["Sequential(1)","Random(2)"], digits=4)

print("\n========== TEST (ENSEMBLE) ==========")
print(f"Best threshold (from VAL): {best_thr:.3f}")
print(f"Accuracy: {acc:.4f}")
print(f"ROC AUC : {auc:.4f}")
print("Confusion matrix [[TN, FP], [FN, TP]]:")
print(cm)
print("\nClassification report:")
print(report)

# Save the model and scaler for future use
import joblib
joblib.dump(models, "xgb_rowlevel_models.pkl")
joblib.dump(scalers, "xgb_rowlevel_scalers.pkl")
print("Models and scalers saved to 'xgb_rowlevel_models.pkl' and 'xgb_rowlevel_scalers.pkl'")
print("Done.")
print("You can load them later using joblib.load()")
print("Example: models = joblib.load('xgb_rowlevel_models.pkl')")
print("Example: scalers = joblib.load('xgb_rowlevel_scalers.pkl')")
print("Make sure to use the same feature engineering and preprocessing steps when loading the models.")
print("Good luck with your XGBoost ensemble!")
print("Remember to tune the parameters further if needed to push accuracy above 80%!")
print("Happy coding!")
print("If you have any questions, feel free to ask!")
print("Thank you for using this script!")
print("Have a great day!")

[VAL] Ensemble | Acc: 0.8181 | AUC: 0.9013 | thr=0.510

========== TEST (ENSEMBLE) ==========
Best threshold (from VAL): 0.510
Accuracy: 0.8216
ROC AUC : 0.9068
Confusion matrix [[TN, FP], [FN, TP]]:
[[21036  3996]
 [ 4937 20095]]

Classification report:
               precision    recall  f1-score   support

Sequential(1)     0.8099    0.8404    0.8249     25032
    Random(2)     0.8341    0.8028    0.8182     25032

     accuracy                         0.8216     50064
    macro avg     0.8220    0.8216    0.8215     50064
 weighted avg     0.8220    0.8216    0.8215     50064

